In [2]:
from difflib import SequenceMatcher
import json
import os
from collections import defaultdict

In [3]:
property = "finance"

# Read dataset

In [4]:
file_path = '競賽資料集/dataset/preliminary/questions_example.json'
with open(file_path, 'r', encoding='utf-8') as file:
    questions = json.load(file)


file_path = '競賽資料集/dataset/preliminary/ground_truths_example.json'
with open(file_path, 'r', encoding='utf-8') as file:
    ground_truths = json.load(file)


file_path = f'output/{property}_kw_extraction.json'
with open(file_path, 'r', encoding='utf-8') as file:
    document_kw_dict = json.load(file)



file_path = f"output/{property}_question_kw.json"
with open(file_path, 'r', encoding='utf-8') as file:
    question_kw_dict = json.load(file)

In [8]:
def get_document_kw(property, document_id):
    document_path = os.path.join(property, str(document_id) + ".pdf")

    merged_dict = defaultdict(list)
    
    for page in document_kw_dict[document_path]:
        temp_kw = page["kw"]
        for key, value in temp_kw.items():
            merged_dict[key].extend(value)
    
    for key in merged_dict:
        merged_dict[key] = list(set(merged_dict[key]))
    
    return merged_dict

In [11]:
get_document_kw(property, 2)

defaultdict(list,
            {'Person': [],
             'Place': ['美國加州',
              '阿拉巴馬州',
              '美國德州',
              '德拉瓦州',
              '馬來西亞',
              '中國',
              '德國',
              '巴西',
              '義大利',
              '日本'],
             'Company': [],
             'Time': ['112年3月31日',
              '111年1至3月',
              '111年3月31日',
              '112年1至3月',
              '111年12月31日'],
             'finance terminology': ['融資',
              '總計',
              '授信額度',
              '帳面價值',
              '短期借款',
              '合約承諾',
              '銀行借款',
              '流動資產',
              '退職後福利',
              '薪資及其他短期員工福利',
              '專利侵權訟',
              '非流動資產']})

# Similariry

In [6]:
def get_similarity(str1, str2):
    similarity = SequenceMatcher(None, str1, str2).ratio()
    return similarity


def cal_kw_similarity(dkw1, dkw2):
    for key in dkw1:
        kw_list1 = dkw1[key]
        kw_list2 = dkw2[key]

    similarity_results = []

    for kw1 in kw_list1:
        max_similarity = -1
        best_match = None

        for kw2 in kw_list2:
            similarity = get_similarity(kw1, kw2)

            if similarity > max_similarity:
                max_similarity = similarity
                best_match = kw2

        if best_match is not None and max_similarity > 0:
            similarity_results.append((kw1, best_match, max_similarity))
    return similarity_results

In [13]:
retieve_result = {}

# temp_questions = [questions["questions"][52]]

for question in questions["questions"]:
    question_wk = question_kw_dict[str(question["qid"])]  # get question kw
    
    if question["category"] == property:
        # print(f"==================qid: {question['qid']}======================")
        # print(question["query"])
        # print(question_wk)
        max_similarity = 0
        max_document = None

        # print("\ndocumment match:")
        for document in question["source"]:
            document_kw = get_document_kw(property, document)
            similarity_list = cal_kw_similarity(question_wk, document_kw)
            
            similarity = sum([i[2] for i in similarity_list]) / 3
            similarity = similarity if similarity < 1 else 1

            # print(document, similarity, similarity_list)

            # 比較當前相似度是否是最大的
            if similarity > max_similarity:
                max_similarity = similarity
                max_document = document

        retieve_result[question["qid"]] = max_document


In [ ]:
ground_truths["ground_truths"]

correct = 0
all = 0
not_noun = 0
for ground_truth in ground_truths["ground_truths"]:
    if ground_truth["category"] == property:
        print("========================")
        print("qid:", ground_truth["qid"])
        print("ground truth retrieve:", ground_truth["retrieve"])
        print("predict retrieve:", retieve_result[ground_truth["qid"]])

        if ground_truth["retrieve"] == retieve_result[ground_truth["qid"]]:
            print("correct")
            correct += 1
        else:
            print("error")

        if retieve_result[ground_truth["qid"]]:
            not_noun += 1
        all += 1

In [148]:
correct / all

0.52

In [149]:
correct / not_noun

0.5777777777777777